# **HOMEWORK 1 - Regressione Lineare**

In questo homework dovrete:

1. Scrivere una funzione di pipeline che deve gestire l' allenamento di un modello di regressione lineare al variare degli iperparametri forniti. Nello specifico:
    * Deve applicare la PCA, se presente.
    
    * Deve applicare la standardizzazione, se presente.

    * Deve applicare la regolarizzazione, se presente.

    * Deve allenare il modello di regressione lineare.

    * Deve calcolare la MAE.

2. Scrivere una funzione che utilizzi la `pipeline` definita al punto 1 e che testi tutte le configurazioni possibili presenti in `configs`. Nel dettaglio la funzione deve:
    * Dividere il dataset in train e validation.

    * Calcolare, grazie alla funzione `pipeline` definita al punto 1, quale configurazione ottiene il punteggio migliore (quale configurazione ha la MAE di validation più bassa).

3. Scrivere una funzione che utilizzi la configurazione migliore prodotta dalla funzione definita al punto 2 e la testi sul test set.

4. Stampare:
    * La migliore configurazione

    * Il miglior MAE di validation

    * Il migliore MAE di train

    * Il MAE di test


Il codice che di seguito trovate già fornito deve essere utilizzato per la risoluzione dell' homework, **NON MODIFICATELO IN ALCUN MODO**.

*testo in corsivo*## **Dataset Wine Quality White**

Il dataset da utilizzare è `wine-quality-white` della libreria `scikit-learn`. Il dataset contiene 11 variabili numeriche + 1 di target che classifica il vino in diverse categorie di qualità. Per il nostro obiettivo la variabile di target è considerata come `float`, permettendoci di applicare la regressione lineare. All' interno del dataset sono contenuti 4898 campioni.

In [ ]:
# Questa cella contiene tutte le librerie di cui necessitate per risolvere l' homework.
# Ricordate di eseguirla prima di iniziare.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [ ]:
hyperparams = {
    # PCA
    'use_pca': [True, False],
    'pca_standardize': [True, False],
    'pca_components': [3, 5, 10],
    # Data standardization
    'data_standardize': [True, False],
    # Regularization l2
    'use_regularization': [True, False],
    'reg_lambda': [0.1, 1, 10],
}

# Calcoliamo tutte le possibili combinazioni di iperparametri
import itertools
combinations = list(itertools.product(*hyperparams.values()))
configs = [dict(zip(hyperparams.keys(), combination)) for combination in combinations]

# Evitiamo le combinazioni non valide
for config in configs:
    if not config['use_pca']:
        config['pca_standardize'] = None
        config['pca_components'] = None
    if not config['use_regularization']:
        config['reg_lambda'] = None
configs = set([tuple(config.items()) for config in configs])

# Convertiamo di nuovo in lista di dizionari
configs = [dict(config) for config in configs]
print(f'Numero di combinazioni: {len(configs)}')

Numero di combinazioni: 56


In `configs` avete una lista di dizionari, ogni dizionario contiene una possibile combinazione di hyperparametri da utilizzare nella fase di training.

In [55]:
# Carica il dataset da OpenML
data = fetch_openml(name='wine-quality-white', version=1, as_frame=True)
X = data.data  # Features del dataset
y = data.target.astype(float)  # Target convertito in float per regressione

def pipeline(X_train, y_train, X_val, y_val, hyperparams):
    """Pipeline completa di preprocessing e regressione lineare"""
    # Copia dei dati per evitare modifiche agli originali
    X_train = X_train.copy()
    X_val = X_val.copy()

    # Sezione PCA: riduzione dimensionalità
    if hyperparams['use_pca']:
        # Standardizzazione prima di PCA se richiesto
        if hyperparams['pca_standardize']:
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)  # Fit solo sul train
            X_val = scaler.transform(X_val)  # Transform su validation

        # Applica PCA con numero componenti specificato
        n_components = hyperparams['pca_components']
        pca = PCA(n_components)
        X_train = pca.fit_transform(X_train)  # Fit solo sul train
        X_val = pca.transform(X_val)  # Transform su validation

    # Standardizzazione generale dei dati se richiesta
    if hyperparams['data_standardize']:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)

    # Aggiunta colonna di 1 per il termine bias
    array_train = np.ones((X_train.shape[0], 1))
    array_validation = np.ones((X_val.shape[0], 1))
    array_train_c = np.c_[array_train, X_train]  # Concatenazione colonna di 1
    array_validation_c = np.c_[array_validation, X_val]

    # Regressione lineare con/senza regolarizzazione L2
    if hyperparams['use_regularization']:
        # Calcolo coefficienti con regolarizzazione
        X_transpose = array_train_c.T
        X_transpose_X = X_transpose @ array_train_c
        I = np.eye(X_transpose_X.shape[0])
        I[0, 0] = 0  # Non regolarizzare il termine bias
        X_transpose_X_inv = np.linalg.inv(X_transpose_X + hyperparams['reg_lambda'] * I)
        beta = X_transpose_X_inv @ X_transpose @ y_train
    else:
        # Calcolo coefficienti senza regolarizzazione
        X_transpose = array_train_c.T
        X_transpose_X_inv = np.linalg.pinv(X_transpose @ array_train_c)
        beta = X_transpose_X_inv @ X_transpose @ y_train

    # Calcolo predizioni e Mean Absolute Error
    y_train_pred = array_train_c @ beta
    y_val_pred = array_validation_c @ beta
    mae_train = np.mean(np.abs(y_train - y_train_pred))  # MAE train
    mae_val = np.mean(np.abs(y_val - y_val_pred))  # MAE validation

    return y_train_pred, y_val_pred, mae_train, mae_val, beta

def Aux_func(X, y, train_split, val_split, test_split, configs):
    """Funzione principale per valutare diverse configurazioni"""
    # Funzione per mescolare i dati
    X, y = shuffle(X, y, random_state=42)

    # Split iniziale 80% (train+val) - 20% test
    n_samples = X.shape[0]
    n_train_val = int(0.8 * n_samples)
    X_train_val = X[:n_train_val]  # 80% iniziale
    y_train_val = y[:n_train_val]
    X_test = X[n_train_val:]  # 20% finale per test
    y_test = y[n_train_val:]

    # Split del train_val in 80% train - 20% validation
    n_train = int(0.8 * X_train_val.shape[0])
    X_train = X_train_val[:n_train]  # 80% di train_val (64% totale)
    y_train = y_train_val[:n_train]
    X_val = X_train_val[n_train:]  # 20% di train_val (16% totale)
    y_val = y_train_val[n_train:]

    # Inizializza variabili per miglior configurazione
    best_config = None
    best_mae_val = float('inf')
    best_mae_train = float('inf')
    best_beta = None
    best_mae_test = float('inf')

    # Valuta tutte le configurazioni di iperparametri
    for config in configs:
        # Valuta su train/validation
        _, _, mae_train, mae_val, beta = pipeline(X_train.copy(), y_train.copy(),
                                               X_val.copy(), y_val.copy(), config)

        # Valuta su test set
        _, _, _, mae_test, _ = pipeline(X_train.copy(), y_train.copy(),
                                      X_test.copy(), y_test.copy(), config)

        # Aggiorna miglior configurazione se necessario
        if mae_val < best_mae_val:
            best_mae_val = mae_val
            best_mae_train = mae_train
            best_config = config
            best_beta = beta
            best_mae_test = mae_test

    return X_train, y_train, X_val, y_val, X_test, y_test, best_config, best_mae_train, best_mae_val, best_beta, best_mae_test

# Esecuzione principale
results = Aux_func(X, y, train_split=0.8, val_split=0.2, test_split=0.2, configs=configs)
X_train, y_train, X_val, y_val, X_test, y_test, best_config, best_mae_train, best_mae_val, best_beta, best_mae_test = results

# Stampa risultati
print(f"\nDimensioni dataset:")
print(f"- Total: {X.shape[0]} campioni")
print(f"- Training: {X_train.shape[0]} ({X_train.shape[0]/X.shape[0]:.1%})")
print(f"- Validation: {X_val.shape[0]} ({X_val.shape[0]/X.shape[0]:.1%})")
print(f"- Test: {X_test.shape[0]} ({X_test.shape[0]/X.shape[0]:.1%})")

print("\nMiglior configurazione:")
for k, v in best_config.items():
    print(f"- {k}: {v}")

print(f"\nMAE training: {best_mae_train:.4f}")
print(f"MAE validation: {best_mae_val:.4f}")
print(f"MAE test: {best_mae_test:.4f}")

# Valutazione finale su train+val completo
X_full_train = np.vstack((X_train, X_val))  # Unisce train e validation
y_full_train = np.concatenate((y_train, y_val))
_, _, _, mae_test_final, _ = pipeline(X_full_train, y_full_train, X_test, y_test, best_config)
print(f"\nMAE test finale (train+val): {mae_test_final:.4f}")


Dimensioni dataset:
- Total: 4898 campioni
- Training: 3134 (64.0%)
- Validation: 784 (16.0%)
- Test: 980 (20.0%)

Miglior configurazione:
- use_pca: True
- pca_standardize: True
- pca_components: 10
- data_standardize: False
- use_regularization: False
- reg_lambda: None

MAE training: 0.5842
MAE validation: 0.5867
MAE test: 0.5968

MAE test finale (train+val): 0.5958


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
